In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [0]:
train_path = "train.csv"
test_path = "test.csv"
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

Y_train = train_data.label
X_train = train_data.drop(columns=["label"])

In [0]:
X_train_nor = X_train / 255.0
test_data_nor = test_data / 255.0

In [19]:
X_train_nor_reshape = X_train_nor.values.reshape(-1, 28, 28 ,1)
X_test_data_nor_reshape = test_data_nor.values.reshape(-1, 28, 28, 1)
print(X_train_nor_reshape.shape)
print(X_test_data_nor_reshape.shape)


(42000, 28, 28, 1)
(28000, 28, 28, 1)


In [0]:
Y_train_categorical = to_categorical(Y_train, num_classes=10)
X_test = X_test_data_nor_reshape

In [0]:
my_model = Sequential()
my_model.add(Conv2D(activation='relu', padding='Same', filters=32, kernel_size=(5,5), input_shape=(28,28,1)))
my_model.add(Conv2D(activation='relu', padding='Same', filters=32, kernel_size=(5,5)))
my_model.add(MaxPool2D(pool_size=(2,2)))
my_model.add(Dropout(0.25))

my_model.add(Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu'))
my_model.add(Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu'))
my_model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
my_model.add(Dropout(0.25))

my_model.add(Flatten())
my_model.add(Dense(265, activation='relu'))
my_model.add(Dropout(0.25))
my_model.add(Dense(10, activation='softmax'))

In [0]:
my_model.compile(optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0), loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
my_model.fit(x=X_train_nor_reshape,y=Y_train_categorical,batch_size =128,epochs=2, verbose=True) 

Epoch 1/2
42000/42000 [==============================] - 328s 8ms/step - loss: 0.0368 - acc: 0.9884
Epoch 2/2
42000/42000 [==============================] - 324s 8ms/step - loss: 0.0321 - acc: 0.9908


In [0]:
results = my_model.predict(X_test)

# select the indix with the maximum probability
results = np.argmax(results,axis=1)

results = pd.Series(results,name="Label")

In [0]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("My_submission.csv",index=False)